<a href="https://colab.research.google.com/github/srilamaiti/spring_2023_w266_final_project_heesuk_iris_srila/blob/main/srila/bertweet_ell.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Installing new libraries**

In [1]:
!pip install nltk
!pip install wordcloud
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 43.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 89.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 5.2 MB/s eta 0:00:00


# **Importing libraries**

In [28]:
import transformers
print(f'transformers version: {transformers.__version__}')
from transformers import logging as hf_logging
from transformers import BertTokenizer, TFBertModel
from transformers import AutoModel, AutoTokenizer
hf_logging.set_verbosity_error()
import nltk
from nltk.tokenize import sent_tokenize
import spacy      
from spacy import displacy
from wordcloud import WordCloud
from wordcloud import STOPWORDS
from wordcloud import ImageColorGenerator
from sklearn.model_selection import train_test_split
nltk.download('punkt')

# Other required libraries
import os
import pandas as pd
import numpy as np
import re
import copy
import sys
import warnings
warnings.filterwarnings("ignore")

# Data visualization libraries
import seaborn as sns
import matplotlib.pyplot as plt

# Tensorflow libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.utils.layer_utils import count_params
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import LearningRateScheduler
from tensorflow.keras.losses import mae
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.losses import binary_crossentropy
from keras.models import Model
from tensorflow.keras import regularizers
from tensorflow.keras.regularizers import l1
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import SGD
from keras.models import load_model

import torch

transformers version: 4.27.2


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# **General functions**

## **Set parameters**

In [3]:
def set_config_param(seed = 99):
    np.random.seed(seed)
    tf.random.set_seed(seed)
    tf.keras.backend.clear_session()
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/MyDrive/Kaggle"
    pd.set_option('display.max_columns', None)
    pd.set_option('display.max_columns', None)
    
    
set_config_param(20230214)

## **Plot loss and accuracy**

In [4]:
def plot_loss_accuracy(history, col_list):
    fig, ax = plt.subplots(2, 6, figsize=(16, 6), sharex='col', sharey='row')
    fig.tight_layout(pad=5.0)
    for idx, col in enumerate(col_list):

        ax[0, idx].plot(history[col + '_loss'], lw=2, color='darkgoldenrod')
        ax[0, idx].plot(history['val_' + col + '_loss'], lw=2, color='indianred')
        #ax[0, idx].legend(loc='center left')
        ax[0, idx].legend(['Train', 'Validation'], fontsize=5)
        ax[0, idx].set_xlabel('Epochs', size=10)
        ax[0, idx].set_title('Loss: ' + col)

        ax[1, idx].plot(history[col + '_accuracy'], lw=2, color='darkgoldenrod')
        ax[1, idx].plot(history['val_' + col + '_accuracy'], lw=2, color='indianred')
        #ax[0, idx].legend(loc='center left')
        ax[1, idx].legend(['Train', 'Validation'], fontsize=5)
        ax[1, idx].set_xlabel('Epochs', size=10)
        ax[1, idx].set_title('Accuracy: ' + col)

## **Custom metric**

In [5]:
def MCRMSE(y_true, y_pred):
    colwise_mse = tf.reduce_mean(tf.square(y_true - y_pred), axis=1)
    return tf.reduce_mean(tf.sqrt(colwise_mse), axis=-1, keepdims=True)

# **Read input files**

In [10]:
input_train_df = pd.read_csv('train.csv')
input_test_df = pd.read_csv('test.csv')
label_cols = df_train.columns[2:]
orig_train_df = copy.deepcopy(input_train_df)
orig_train_df.head()

,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0016926B079C,I think that students would benefit from learn...,3.5,3.5,3.0,3.0,4.0,3.0
1,0022683E9EA5,When a problem is a change you have to let it ...,2.5,2.5,3.0,2.0,2.0,2.5
2,00299B378633,"Dear, Principal\n\nIf u change the school poli...",3.0,3.5,3.0,3.0,3.0,2.5
3,003885A45F42,The best time in life is when you become yours...,4.5,4.5,4.5,4.5,4.0,5.0
4,0049B1DF5CCC,Small act of kindness can impact in other peop...,2.5,3.0,3.0,3.0,2.5,2.5


# **Spliting the data**
Original test data is very limited, there are only 3 records and it does not have labels to test. So we decided to repurpose the given train data to split into train, test and validation sets.

In [9]:
shuffle = np.random.permutation(np.arange(orig_train_df.shape[0]))
orig_train_df = orig_train_df.iloc[shuffle]
split=(0.8,0.1,0.1)
splits = np.multiply(len(orig_train_df), split).astype(int)
df_train, df_val, df_test = np.split(orig_train_df, [splits[0], splits[0] + splits[1]])

# **Model building**

In [19]:
set_config_param()
MAX_LENGTH = 512
epochs = 5
batch_size = 8
dropout = .1
learning_rate = .00005
number_of_hidden_layer = 1
hidden_layer_node_count = 256
trainable_flag = False
retrain_layer_count = 0

In [25]:
train_text = df_train['full_text'].tolist()
test_text = df_test['full_text'].tolist()
val_text = df_val['full_text'].tolist()

In [29]:
bertweet_model_checkpoint = 'vinai/bertweet-base'   
bertweet_model_v1 = AutoModel.from_pretrained(bertweet_model_checkpoint)
bertweet_tokenizer = AutoTokenizer.from_pretrained(bertweet_model_checkpoint, use_fast=False, normalization = True)

https://colab.research.google.com/github/devhemza/BERTweet_sentiment_analysis/blob/main/BERTweet.ipynb#scrollTo=KrZkll3WI1jF

In [ ]:
class ell_dataset(Dataset):

  def __init__(self, tweets, targets, tokenizer, max_len):
    self.tweets = tweets
    self.targets = targets
    self.tokenizer = tokenizer
    self.max_len = max_len
  
  def __len__(self):
    return len(self.tweets)
  
  def __getitem__(self, item):
    tweet = str(self.tweets[item])
    target = self.targets[item]

    encoding = self.tokenizer.encode_plus(
      tweet,
      add_special_tokens=True,
      max_length=self.max_len,
      truncation= True,
      return_token_type_ids=False,
      padding = 'max_length',
      return_attention_mask=True,
      return_tensors='pt',
    )

    return {
      'tweet_text': tweet,
      'input_ids': encoding['input_ids'].flatten(),
      'attention_mask': encoding['attention_mask'].flatten(),
      'targets': torch.tensor(target, dtype=torch.long)
    }

In [21]:
input_ids = tf.keras.layers.Input(shape=(MAX_LENGTH,), dtype=tf.int64, name='input_ids_layer')
token_type_ids = tf.keras.layers.Input(shape=(MAX_LENGTH,), dtype=tf.int64, name='token_type_ids_layer')
attention_mask = tf.keras.layers.Input(shape=(MAX_LENGTH,), dtype=tf.int64, name='attention_mask_layer')

bert_inputs = {'input_ids': input_ids,
               'token_type_ids': token_type_ids,
               'attention_mask': attention_mask
              }
                   
# Bert output: being used as an input feature in the classification model below
bertweet_model_v1_out = bertweet_model_v1(bert_inputs)        
# pooler_output = bert_out[1]             # one vector for each
cls_token = bertweet_model_v1_out[0][:, 0, :]          # give us a raw CLS tokens

layer_list = []

for hidden_layer_number in range(number_of_hidden_layer):
    if hidden_layer_number == 0:
        hidden_layer = tf.keras.layers.Dense(units = hidden_layer_node_count
                                    , activation = 'relu'
                                    , name = 'hidden_layer_' + str(hidden_layer_number + 1)
                                     )(cls_token)
    else:
        hidden_layer = tf.keras.layers.Dense(units = hidden_layer_node_count
                                    , activation = 'relu'
                                    , name = 'hidden_layer_' + str(hidden_layer_number + 1)
                                     )(layer_list[-1])
    layer_list.append(hidden_layer)
    dropout_layer = tf.keras.layers.Dropout(dropout, name = 'dropout_layer_' + str(hidden_layer_number + 1))(hidden_layer) 
    layer_list.append(dropout_layer)

output = tf.keras.layers.Dense(6,)(layer_list[-1])
bertweet_v1_regression_model = tf.keras.Model(inputs = [input_ids, 
                                                        token_type_ids,
                                                        attention_mask
                                                       ], 
                                                       outputs = output
                                             )

bertweet_v1_regression_model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate),
                                     loss = MCRMSE, 
                                     metrics=MCRMSE
                                    ) 
print(bertweet_v1_regression_model.summary())



AttributeError: ignored

In [ ]:
keras.utils.plot_model(bertweet_v1_regression_model, show_shapes=False, show_dtype=False, show_layer_names=True, dpi=90)

In [20]:
bertweet_v1_regression_model_history = bertweet_v1_regression_model.fit([train_encodings.input_ids, 
                                                                         train_encodings.token_type_ids, 
                                                                         train_encodings.attention_mask
                                                                        ], 
                                                                        y_train,   
                                                                        validation_data =([val_encodings.input_ids, 
                                                                                           val_encodings.token_type_ids, 
                                                                                           val_encodings.attention_mask
                                                                                          ], 
                                                                                          y_val
                                                                                         ),    
                                                                        batch_size = batch_size, 
                                                                        epochs = epochs
                                                                       )                                                  
bertweet_v1_regression_model_history_df = pd.DataFrame(bertweet_v1_regression_model_history.history)
bertweet_v1_regression_model_history_df.T


NameError: ignored

In [ ]:
score_bertweet_v1_regression_model = bertweet_v1_regression_model.evaluate([test_encodings.input_ids, 
                                                                            test_encodings.token_type_ids, 
                                                                            test_encodings.attention_mask
                                                                           ], 
                                                                           y_test
                                                                          ) 

print(score_bertweet_v1_regression_model)

In [ ]:
predictions_bertweet_v1_regression_model = bertweet_v1_regression_model.predict([test_encodings.input_ids, 
                                                                                 test_encodings.token_type_ids, 
                                                                                 test_encodings.attention_mask
                                                                                ]
                                                                               )
print(predictions_bertweet_v1_regression_model)

In [ ]:
df_test[label_cols]

In [ ]:
predictions_bertweet_v1_regression_model

In [ ]:
x_arr = np.arange(len(bertweet_v1_regression_model['loss'])) + 1

fig = plt.figure(figsize=(12, 4))
ax = fig.add_subplot(1, 2, 1)
ax.plot(x_arr, bertweet_v1_regression_model_history_df['loss'], '-o', label='Train loss')
ax.plot(x_arr, bertweet_v1_regression_model_history_df['val_loss'], '--<', label='Validation loss')
ax.legend(fontsize=15)
ax.set_xlabel('Epoch', size=15)
ax.set_ylabel('Loss', size=15)

ax = fig.add_subplot(1, 2, 2)
ax.plot(x_arr, bertweet_v1_regression_model_history_df['MCRMSE'], '-o', label='Train MCRMSE')
ax.plot(x_arr, bertweet_v1_regression_model_history_df['val_MCRMSE'], '--<', label='Validation MCRMSE')
ax.legend(fontsize=15)
ax.set_xlabel('Epoch', size=15)
ax.set_ylabel('Accuracy', size=15)
ax.set_ylim(0,1)
plt.show()